<a href="https://colab.research.google.com/github/cjfghk5697/multi-label-classification-Mnist/blob/main/Add_Scheduler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 이진화 코드 추가
 이진화란 이미지를 흰색과 검은색으로 바꾼다고 생각하면 된다.
속도가 향상되고 정확도도 미약하지만 향상된것을 볼수 있다.

 [Dacon 적용 예시](https://dacon.io/competitions/official/235697/codeshare/2436?page=2&dtype=recent)


 [이진화 설명](https://gmnam.tistory.com/263)

 <hr>

 # 2. train, valid set 나누기
```
train_dataset, valid_dataset = torch.utils.data.random_split(trainset, [train_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=64, num_workers=8)
valid_loader = DataLoader(valid_dataset, batch_size=64, num_workers=8)
```
처음에는 나누어지지 않아있는 코드였단 걸 발견하고 random_split 함수를 이용해 나눔. 그리고 학습 부분에서 valid 데이터를 enumerate하는 구문추가해서 valid 확인.

<hr>

# 3. Multi Sample Dropout 구현

[자료](https://velog.io/@sujeongim/%EC%83%88%EB%A1%9C-%EB%B0%B0%EC%9A%B4-%EC%BD%94%EB%93%9C-%EC%A1%B0%EA%B0%81-Multi-Sample-Dropout)
단순 dropout 기법을 안썼다. 근데 큰효과가 있는지는 잘 모르지만 overfitting의 위협도 없앨겸 선택했다.

<hr>

# 4. Scheduler 추가
일단 간단하게 StepLR scheduler을 추가했다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#%cd /content/drive/MyDrive/Test image2

#!unzip -qq "/content/drive/MyDrive/test_dirty_mnist_2nd.zip"

%cd /content/drive/MyDrive/Train image2
!unzip -qq "/content/drive/MyDrive/dirty_mnist_2nd.zip"

/content/drive/MyDrive/Train image2


In [ ]:
!pip3 install torchinfo
!pip3 install cv2

In [ ]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
from PIL import Image
from torchvision import transforms
from torchvision.models import resnet50
#이진화 추가
import cv2
from PIL import Image


## 1. 커스텀 데이터셋 만들기

In [ ]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(
                self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)


        image = np.array(image)
#이진화 완료 cv2 툴을 써서 threshold로 254 값으로 변환. cv2는 넘파이만 사용 가능해서 넘파이로 변환후 fromarry로 pil로 바꾼다.
        image_th = cv2.threshold(image,254,255,0)[1]
        image_th = Image.fromarray(image_th) # NumPy array to PIL image
        if self.transforms is not None:
            image = self.transforms(image_th)

        return image, target

## 2. 이미지 어그멘테이션

In [ ]:
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [ ]:
trainset = MnistDataset('/content/drive/MyDrive/Train image2', '/content/drive/MyDrive/dirty_mnist_2nd_answer.csv', transforms_train)
testset = MnistDataset('/content/drive/MyDrive/Test image2', '/content/drive/MyDrive/sample_submission.csv', transforms_test)

train_size = int(0.8 * len(trainset))
valid_size = len(trainset) - train_size

train_dataset, valid_dataset = torch.utils.data.random_split(trainset, [train_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=32, num_workers=8)
valid_loader = DataLoader(valid_dataset, batch_size=16, num_workers=8)

test_loader = DataLoader(testset, batch_size=32, num_workers=4)

## 3. ResNet50 모형

In [ ]:
class MnistModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = resnet50(pretrained=True)
        self.classifier = nn.Linear(1000, 26)
         # dropout samples 생성 
        self.dropouts = nn.ModuleList([
            nn.Dropout(0.5) for _ in range(5)
        ])

    def forward(self, x):
        x = self.resnet(x)

#multi sample Dropout 구현. 
        for i,dropout in enumerate(self.dropouts):
            if i== 0:
                out = dropout(x.clone())
                out = self.classifier(out)
            else:
                temp_out = dropout(x.clone())
                out += self.classifier(temp_out)
        return torch.sigmoid(out/len(self.dropouts))
        x = self.classifier(x)

        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MnistModel().to(device)
print(summary(model, input_size=(1, 3, 256, 256), verbose=0))

## 4. 학습하기

In [ ]:

optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MultiLabelSoftMarginLoss()
    #scheduler 추가 및 step 추가

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size = 5,
                                                gamma = 0.75)
num_epochs = 10
model.train()

for epoch in range(num_epochs):
    train_acc_list = []
    valid_acc_list =[]
    for i, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
            train_acc_list.append(acc)

            print(f'{epoch}: "train loss:"{loss.item():.5f}, "train acc: "{acc.item():.5f}')
    for i, (images, targets) in enumerate(valid_loader):
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)
        valid_loss = criterion(outputs, targets)
      
        if (i+1) % 10 == 0:
            outputs = outputs > 0.5
            valid_acc = (outputs == targets).float().mean()
            valid_acc_list.append(acc)

            print(f'{epoch}: "val loss:"{valid_loss.item():.5f}, "val acc: "{valid_acc.item():.5f}')
    lr_scheduler.step()


## 5. 추론하기

In [ ]:
submit = pd.read_csv('data/sample_submission.csv')

model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    outputs = outputs > 0.5
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.long().squeeze(0).detach().cpu().numpy()
    
submit.to_csv('submit.csv', index=False)

In [ ]:
import matplotlib as plt

plt.plot(train_acc_list)
